## Weakly Supervised Instance Segmentation using Class Peak Response 
### Evaluation code of object counting in Pascal

In [1]:
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

import os
import json
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from nest import modules, run_tasks
import matplotlib.pyplot as plt
import random
import xml.etree.ElementTree as ET
from collections import OrderedDict
from PIL import Image
import cv2

In [2]:
def mrmse(non_zero,count_pred, count_gt):
    ## compute mrmse
    nzero_mask=torch.ones(count_gt.size())
    if non_zero==1:
        nzero_mask=torch.zeros(count_gt.size())
        nzero_mask[count_gt!=0]=1
    mrmse=torch.pow(count_pred - count_gt, 2)
    mrmse = torch.mul(mrmse, nzero_mask)
    mrmse = torch.sum(mrmse, 0)
    nzero = torch.sum(nzero_mask, 0)
    mrmse = torch.div(mrmse, nzero)
    mrmse = torch.sqrt(mrmse)
    mrmse = torch.mean(mrmse)
    return mrmse

def rel_mrmse(non_zero,count_pred, count_gt):
    ## compute reltive mrmse
    nzero_mask=torch.ones(count_gt.size())
    if non_zero==1:
        nzero_mask=torch.zeros(count_gt.size())
        nzero_mask[count_gt!=0]=1
    num = torch.pow(count_pred - count_gt, 2)
    denom = count_gt.clone()
    denom = denom+1
    rel_mrmse = torch.div(num, denom)
    rel_mrmse = torch.mul(rel_mrmse, nzero_mask)
    rel_mrmse = torch.sum(rel_mrmse, 0)
    nzero = torch.sum(nzero_mask, 0)
    rel_mrmse = torch.div(rel_mrmse, nzero)
    rel_mrmse = torch.sqrt(rel_mrmse)
    rel_mrmse = torch.mean(rel_mrmse)
    return rel_mrmse

def return_count_obj_rm_diff(xml_file,class_name):
    count=0
    tree = ET.parse(xml_file)
    objs = tree.findall('object')
    for ix, obj in enumerate(objs):
        if obj.find('name').text==class_name and int(obj.find('difficult').text)==0:
            count+=1
    return count

In [3]:
# object categories
class_names = modules.pascal_voc_object_categories()
image_size = 448
# image pre-processor
transformer = modules.image_transform(
    image_size = [image_size, image_size],
    augmentation = dict(),
    mean = [0.485, 0.456, 0.406],
    std = [0.229, 0.224, 0.225])

In [4]:
## set pascal 2007 dataset path 
pascal_path='/raid/xiaoke/pascal_2007/VOCdevkit/VOC2007'

## load pascal 2007 test image list
with open(pascal_path+'/ImageSets/Main/test.txt','r') as f:
    image_list=[]
    for ima in f:
        ima=ima.strip('\n')
        image_list.append(ima)

## get the ground truth count
class_names = modules.pascal_voc_object_categories()
num_classes=len(class_names)
class_labels = OrderedDict()
for class_idx in range(num_classes):
    filename = os.path.join(
        pascal_path+'/ImageSets/Main/', class_names[class_idx] + '_' + 'test' + '.txt')
    with open(filename, 'r') as f:
        for line in f:
            name, label = line.split()
            if name not in class_labels:
                class_labels[name] = np.zeros(num_classes)
            class_labels[name][class_idx] = int(label)
            if int(label)!=-1:
                count=return_count_obj_rm_diff(os.path.join(pascal_path+'/Annotations/',name+'.xml'),
                    class_names[class_idx])
                class_labels[name][class_idx] = int(count)
            else:
                class_labels[name][class_idx] = int(0)
gt_count=[]
for i in list(class_labels.items()):
    gt_count.append(i[1])
gt_count=np.array(gt_count)

In [6]:
backbone = modules.fc_resnet50(channels=60, pretrained=False)
model = modules.peak_response_mapping(backbone,enable_peak_stimulation=True,peak_stimulation='addedmodule5',
                                     sub_pixel_locating_factor=1)
# loaded pre-trained weights
model = nn.DataParallel(model)
state = torch.load('../models/counting/pascal07.pt')
model.load_state_dict(state['model'])
model = model.module.cuda()
model = model.eval()
pred_count=[]
for index_d,ima in enumerate(image_list):
    if index_d%500==0:
        print(index_d)
    raw_img = Image.open(pascal_path+'/JPEGImages/'+str(ima)+'.jpg').convert('RGB')
    width, height=raw_img.size
    input_var = transformer(raw_img).unsqueeze(0).cuda().requires_grad_()
    confidence,class_response_map1,peak = model(input_var,1)
    confidence=confidence.cpu().detach().numpy()
    count_one = F.adaptive_avg_pool2d(class_response_map1, 1).squeeze(2).squeeze(2).detach().cpu().numpy()[0]
    confidence[confidence<0]=0
    confidence=confidence[0]
    confidence[confidence>0]=1
    pred_count.append(np.round(confidence*count_one))
pred_count=np.array(pred_count)
print(mrmse(1,torch.from_numpy(pred_count).float(), torch.from_numpy(gt_count).float()))
print(rel_mrmse(1,torch.from_numpy(pred_count).float(), torch.from_numpy(gt_count).float()))
print(mrmse(0,torch.from_numpy(pred_count).float(), torch.from_numpy(gt_count).float()))
print(rel_mrmse(0,torch.from_numpy(pred_count).float(), torch.from_numpy(gt_count).float()))

<function PeakResponseMapping._median_filter at 0x7efc65170048>
addedmodule5
enable_peak_stimulation on
0
500
1000
1500
2000
2500
3000
3500
4000
4500
tensor(1.1474)
tensor(0.6134)
tensor(0.2991)
tensor(0.1836)
